### PARTICLE SWARM 

In [ ]:
import numpy as np
np.set_printoptions(suppress=True)

In [ ]:
%run fuzzy-pakar-jarak-z.ipynb

In [ ]:
class Particle:
    def __init__(self, bounds):
        self.bounds = bounds
        self.position = self._generate_position()
        self.velocity = self._generate_velocity()
        self.best_position = np.copy(self.position)
        self.best_score = -float('inf')

    def _generate_position(self):
        position = []
        for min_bound, max_bound, length in self.bounds:
            segment = np.random.uniform(min_bound, max_bound, size=length)
            position.extend(segment)
        return np.array(position)

    def _generate_velocity(self):   
        velocity = []
        for _, _, length in self.bounds:
            segment = np.random.uniform(-1, 1, size=length)
            velocity.extend(segment)
        return np.array(velocity)

    def update_velocity(self, global_best_position, w, c1, c2):
        r1 = np.random.rand(len(self.position))
        r2 = np.random.rand(len(self.position))
        cognitive = c1 * r1 * (self.best_position - self.position)
        social = c2 * r2 * (global_best_position - self.position)
        self.velocity = w * self.velocity + cognitive + social

    def update_position(self):
        self.position += self.velocity
        start_idx = 0
        for min_bound, max_bound, length in self.bounds:
            end_idx = start_idx + length
            self.position[start_idx:end_idx] = np.clip(
                self.position[start_idx:end_idx], min_bound, max_bound
            )
            start_idx = end_idx

In [ ]:
def objective_function(posisi_partikel, dataset):
    total_error = 0
    for i in range(len(dataset)):
        waktu_belajar = dataset.loc[i, 'rerata_durasi_akses_materi']
        lama_jawab_soal = dataset.loc[i, 'durasi_berlatih']
        nilai = dataset.loc[i, 'nilai']
        kepercayaan_pakar = dataset.loc[i, 'kepercayaan_pakar']
        # fuzzifikasi
        x, y, z = func_fuzzifikasi(waktu_belajar, lama_jawab_soal, nilai, posisi_partikel)
        # inferensi mamdani
        hasil_inferensi, rule_text = inferensi_mamdani(x, y, z, rules)
        # agregasi
        hasil_agregasi = agregasi(hasil_inferensi)
        # defuzzifikasi coa
        hasil_defuzzifikasi = defuzzifikasi(hasil_agregasi)
        # menghitung error
        nilai_error = error_kelulusan(hasil_defuzzifikasi, kepercayaan_pakar)
        total_error += nilai_error
    # hitung akurasi
    akurasi = 100 - (total_error / len(dataset))
    indices = [0, 4, 5, 9, 10, 14]
    if any(posisi_partikel[i] == 0 for i in indices):
        akurasi = 0
    return akurasi

In [ ]:
def ensure_constraints(posisi_partikel):
    posisi_partikel[1:4] = sorted(posisi_partikel[1:4])
    posisi_partikel[6:9] = sorted(posisi_partikel[6:9])
    posisi_partikel[11:14] = sorted(posisi_partikel[11:14])

In [ ]:
def particle_swarm_optimization(bounds, dataset, num_particles, max_iter, w=0.5, c1=2, c2=2):
    swarm = []
    temp= []
    for _ in range(num_particles):
        swarm.append(Particle(bounds))
    global_best_position = None
    global_best_score = -float('inf')

    for iteration in range(max_iter):
        for particle in swarm:
            ensure_constraints(particle.position)
            particle_score = objective_function(particle.position, dataset)
            # print(particle_score)
            if particle_score > particle.best_score:
                particle.best_score = particle_score
                particle.best_position = np.copy(particle.position)

            if particle_score > global_best_score:
                global_best_score = particle_score
                global_best_position = np.copy(particle.position)

        # Update velocity and position
        for particle in swarm:
            particle.update_velocity(global_best_position, w, c1, c2)
            particle.update_position()
                # ensure_constraints(particle.position)
        temp.append(global_best_score)
            
        print(f"Iteration {iteration+1}/{max_iter}, Best Score: {global_best_score}")
        
    return global_best_position, global_best_score, temp

In [ ]:
bounds = [
    (0, 700, 5),  # Elemen 0-10 dengan batas [0, 1]
    (0, 7500, 5),  # Elemen 11-20 dengan batas [1, 5]
    (0, 100, 5)  # Elemen 21-31 dengan batas [-3, 3]
]

# Run PSO
num_particles = 50
max_iter = 50
best_params, best_score, best_score_hist = particle_swarm_optimization(bounds, dataset, num_particles, max_iter)

In [ ]:
print(best_params)
print(f"\nBest Accuracy: {best_score:.4f}")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(best_score_hist, color="red")
plt.title("Akurasi setiap Iterasi", fontsize=16)
plt.xlabel("Iterasi ke-", fontsize=12)
plt.ylabel("Akurasi", fontsize=12)
plt.show()